# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

In [2]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.sklearn import SKLearn

ws = Workspace.get(name="quick-starts-ws-127323")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_DS3_V2"
compute_name="notebook127323"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())





Workspace name: quick-starts-ws-127323
Azure region: southcentralus
Subscription id: c1dcf523-8ab0-429f-8dba-143a6c49ec84
Resource group: aml-quickstarts-127323


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
#ws = Workspace.from_config()

# choose a name for experiment
#experiment_name = 'your experiment name here'

#experiment=Experiment(ws, experiment_name)

#workspace = Workspace(subscription_id, resource_group, workspace_name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./', target_path='data')

# create a dataset referencing the cloud location
#

Uploading an estimated of 3 files
Target already exists. Skipping upload for data/Auto_ML_version_1.ipynb
Target already exists. Skipping upload for data/.ipynb_checkpoints/Auto_ML_version_1-checkpoint.ipynb
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 2
Uploaded 1 files


$AZUREML_DATAREFERENCE_09a9cdd1b95e4b5cb62a54939bbb2137

In [5]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/heart_failure_clinical_records_dataset.csv'))])

In [16]:
dataset.to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.00,0,61,1,38,1,155000.00,1.10,143,1,1,270,0
295,55.00,0,1820,0,38,0,270000.00,1.20,139,0,0,271,0
296,45.00,0,2060,1,60,0,742000.00,0.80,138,0,0,278,0
297,45.00,0,2413,0,38,0,140000.00,1.40,140,1,1,280,0


In [ ]:
"""
est = SKLearn(source_directory="./", 
                compute_target=compute_target, 
                vm_size=vm_size,
                entry_script='train.py'
               )

"""

In [ ]:
dataset

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here



from azureml.train.automl import AutoMLConfig
import numpy as np


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=compute_target,
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name='DEATH_EVENT',
    #allowed_models=["LightGBM","RandomForest"],
    n_cross_validations=3)


In [8]:
exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib
from azureml.widgets import RunDetails
RunDetails(automl_model).show()
best_run, fitted_model = automl_model.get_output()
print(best_run)
print(fitted_model)
from azureml.core.model import Model
import joblib
joblib.dump(fitted_model,"automl_best_model.pkl")
model = Model.register(workspace=ws, model_path="automl_best_model.pkl", model_name="test")

Running on remote.
Running on remote compute: notebook127323
Parent Run ID: AutoML_a47fe669-af73-4a01-aeb1-3cb05b9c9f72

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: automl_test,
Id: AutoML_a47fe669-af73-4a01-aeb1-3cb05b9c9f72_33,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_decrease=0.0,
                                                                                                    min_impurity_split=None,
               

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "AutoML-AzureML-AutoML",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "cond

In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [43]:
input_data=x_df[0:2].to_dict(orient='records')

In [47]:
service.scoring_uri

'http://e5401dfb-1add-43be-987a-87e391a83eb3.southcentralus.azurecontainer.io/score'

In [52]:
input_data={"data":[{'age': 75.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 582,
  'diabetes': 0,
  'ejection_fraction': 20,
  'high_blood_pressure': 1,
  'platelets': 265000.0,
  'serum_creatinine': 1.9,
  'serum_sodium': 130,
  'sex': 1,
  'smoking': 0,
  'time': 4}]}

In [59]:
i=json.dumps(input_data)

In [64]:
with open("data.json","w") as _f:
    _f.write(i)
    

In [105]:
headers={"Content-Type":"application/json"}
key="nmzf01o9jlalITeEPXejCGkw7Bqs0Iv0"
headers["Authorization"]=f'Bearer "R7PFVjyegEIU16PFDI0odluGw14bLdvY"'

In [108]:
import requests
result=requests.post('http://e5401dfb-1add-43be-987a-87e391a83eb3.southcentralus.azurecontainer.io/score',i,headers=headers)

In [109]:
result.content


b'Unauthorized, no token matched'

In [ ]:

service.delete()